In [57]:
import os
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [58]:

# Define the base folder
base_folder = 'Multimodal_images'

# Use glob to find all jpg files recursively
jpg_files = glob.glob(os.path.join(base_folder, '**', '*.jpg'), recursive=True)

# Print the list of jpg file paths
print(jpg_files)


['Multimodal_images\\cyanosis\\Image_1.jpg', 'Multimodal_images\\cyanosis\\Image_10.jpg', 'Multimodal_images\\cyanosis\\Image_11.jpg', 'Multimodal_images\\cyanosis\\Image_13.jpg', 'Multimodal_images\\cyanosis\\Image_14.jpg', 'Multimodal_images\\cyanosis\\Image_18.jpg', 'Multimodal_images\\cyanosis\\Image_19.jpg', 'Multimodal_images\\cyanosis\\Image_2.jpg', 'Multimodal_images\\cyanosis\\Image_20.jpg', 'Multimodal_images\\cyanosis\\Image_29.jpg', 'Multimodal_images\\cyanosis\\Image_3.jpg', 'Multimodal_images\\cyanosis\\Image_31.jpg', 'Multimodal_images\\cyanosis\\Image_32.jpg', 'Multimodal_images\\cyanosis\\Image_37.jpg', 'Multimodal_images\\cyanosis\\Image_4.jpg', 'Multimodal_images\\cyanosis\\Image_43.jpg', 'Multimodal_images\\cyanosis\\Image_5.jpg', 'Multimodal_images\\cyanosis\\Image_6.jpg', 'Multimodal_images\\dry scalp\\Image_13.jpg', 'Multimodal_images\\dry scalp\\Image_18.jpg', 'Multimodal_images\\dry scalp\\Image_19.jpg', 'Multimodal_images\\dry scalp\\Image_22.jpg', 'Multimodal

In [59]:
len(jpg_files)

532

In [60]:
# Function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    labels = []
    for path in image_paths:
        image = load_img(path, target_size=target_size)   #module come from tensorflow.keras.preprocessing.image
        image = img_to_array(image)  # (height x width x channels) = (224,224,3)
        images.append(image)
        label = os.path.basename(os.path.dirname(path))  # Assuming folder name is the label
        labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [61]:
jpg_files

['Multimodal_images\\cyanosis\\Image_1.jpg',
 'Multimodal_images\\cyanosis\\Image_10.jpg',
 'Multimodal_images\\cyanosis\\Image_11.jpg',
 'Multimodal_images\\cyanosis\\Image_13.jpg',
 'Multimodal_images\\cyanosis\\Image_14.jpg',
 'Multimodal_images\\cyanosis\\Image_18.jpg',
 'Multimodal_images\\cyanosis\\Image_19.jpg',
 'Multimodal_images\\cyanosis\\Image_2.jpg',
 'Multimodal_images\\cyanosis\\Image_20.jpg',
 'Multimodal_images\\cyanosis\\Image_29.jpg',
 'Multimodal_images\\cyanosis\\Image_3.jpg',
 'Multimodal_images\\cyanosis\\Image_31.jpg',
 'Multimodal_images\\cyanosis\\Image_32.jpg',
 'Multimodal_images\\cyanosis\\Image_37.jpg',
 'Multimodal_images\\cyanosis\\Image_4.jpg',
 'Multimodal_images\\cyanosis\\Image_43.jpg',
 'Multimodal_images\\cyanosis\\Image_5.jpg',
 'Multimodal_images\\cyanosis\\Image_6.jpg',
 'Multimodal_images\\dry scalp\\Image_13.jpg',
 'Multimodal_images\\dry scalp\\Image_18.jpg',
 'Multimodal_images\\dry scalp\\Image_19.jpg',
 'Multimodal_images\\dry scalp\\Image

In [62]:
# Load the images and their labels
image_paths =jpg_files
images, labels = load_and_preprocess_images(image_paths)


In [63]:
labels

array(['cyanosis', 'cyanosis', 'cyanosis', 'cyanosis', 'cyanosis',
       'cyanosis', 'cyanosis', 'cyanosis', 'cyanosis', 'cyanosis',
       'cyanosis', 'cyanosis', 'cyanosis', 'cyanosis', 'cyanosis',
       'cyanosis', 'cyanosis', 'cyanosis', 'dry scalp', 'dry scalp',
       'dry scalp', 'dry scalp', 'dry scalp', 'dry scalp', 'dry scalp',
       'dry scalp', 'dry scalp', 'dry scalp', 'dry scalp', 'dry scalp',
       'dry scalp', 'dry scalp', 'dry scalp', 'dry scalp', 'dry scalp',
       'dry scalp', 'dry scalp', 'edema', 'edema', 'edema', 'edema',
       'edema', 'edema', 'edema', 'edema', 'edema', 'edema', 'edema',
       'edema', 'edema', 'edema', 'eye_inflamation', 'eye_inflamation',
       'eye_inflamation', 'eye_inflamation', 'eye_inflamation',
       'eye_inflamation', 'eye_inflamation', 'eye_inflamation',
       'eye_inflamation', 'eye_inflamation', 'eye_inflamation',
       'eye_inflamation', 'eye_inflamation', 'eye_inflamation',
       'eye_inflamation', 'eye_inflamation', 'e

In [64]:
images

array([[[[  9.,  13.,  12.],
         [  6.,  10.,   9.],
         [  7.,  11.,  10.],
         ...,
         [177., 226., 205.],
         [182., 227., 207.],
         [181., 230., 209.]],

        [[  6.,  10.,   9.],
         [  4.,   8.,   7.],
         [  6.,  10.,   9.],
         ...,
         [177., 226., 205.],
         [182., 227., 207.],
         [181., 230., 209.]],

        [[  7.,  11.,  10.],
         [  5.,   9.,   8.],
         [  5.,   9.,   8.],
         ...,
         [179., 228., 207.],
         [180., 225., 205.],
         [179., 228., 207.]],

        ...,

        [[103., 168., 206.],
         [ 92., 156., 192.],
         [ 91., 155., 193.],
         ...,
         [206., 171., 149.],
         [201., 174., 147.],
         [200., 173., 144.]],

        [[ 66., 135., 168.],
         [ 55., 121., 155.],
         [ 33.,  93., 129.],
         ...,
         [203., 171., 148.],
         [206., 176., 152.],
         [201., 175., 150.]],

        [[ 59., 125., 159.],
       

In [65]:

images.shape

(532, 224, 224, 3)

In [66]:
# Preprocess images for ResNet50
images = tf.keras.applications.resnet50.preprocess_input(images) 
#For ResNet50, this involves converting the pixel values from a range of [0, 255] to a range suitable for the model.

In [79]:
images

array([[[[ -91.939     , -103.779     , -114.68      ],
         [ -94.939     , -106.779     , -117.68      ],
         [ -93.939     , -105.779     , -116.68      ],
         ...,
         [ 101.061     ,  109.221     ,   53.32      ],
         [ 103.061     ,  110.221     ,   58.32      ],
         [ 105.061     ,  113.221     ,   57.32      ]],

        [[ -94.939     , -106.779     , -117.68      ],
         [ -96.939     , -108.779     , -119.68      ],
         [ -94.939     , -106.779     , -117.68      ],
         ...,
         [ 101.061     ,  109.221     ,   53.32      ],
         [ 103.061     ,  110.221     ,   58.32      ],
         [ 105.061     ,  113.221     ,   57.32      ]],

        [[ -93.939     , -105.779     , -116.68      ],
         [ -95.939     , -107.779     , -118.68      ],
         [ -95.939     , -107.779     , -118.68      ],
         ...,
         [ 103.061     ,  111.221     ,   55.32      ],
         [ 101.061     ,  108.221     ,   56.32      ],
  

In [80]:
images.shape

(532, 224, 224, 3)

In [85]:
# Encode labels
label_encoder = LabelEncoder()
print(label_encoder)
#For example, if labels are ['cat', 'dog', 'mouse'], it might convert them to [0, 1, 2]
labels_encoded = label_encoder.fit_transform(labels) 
print(labels_encoded)
#Convert Numerical Labels to One-Hot Encoding:
labels_categorical = to_categorical(labels_encoded)
labels_categorical

LabelEncoder()
[0]


array([[1.]], dtype=float32)

In [68]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)


In [69]:
# Load ResNet50 model without the top fully connected layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [70]:
len(label_encoder.classes_)

18

In [71]:
# Add custom layers on top of the base model
model = Sequential([
    base_model,
    GlobalMaxPooling2D(),
    Dense(1024, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [72]:
# Freeze the base model layers to avoid retraining them
base_model.trainable = False

In [73]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [74]:
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
14/14 [==============================] - 42s 3s/step - loss: 25.3554 - accuracy: 0.2094 - val_loss: 30.9146 - val_accuracy: 0.3458
Epoch 2/10
14/14 [==============================] - 34s 2s/step - loss: 6.7082 - accuracy: 0.6000 - val_loss: 11.0438 - val_accuracy: 0.5421
Epoch 3/10
14/14 [==============================] - 34s 2s/step - loss: 2.1323 - accuracy: 0.7600 - val_loss: 2.4562 - val_accuracy: 0.6168
Epoch 4/10
14/14 [==============================] - 35s 3s/step - loss: 0.5189 - accuracy: 0.8729 - val_loss: 2.5768 - val_accuracy: 0.4860
Epoch 5/10
14/14 [==============================] - 34s 2s/step - loss: 0.2378 - accuracy: 0.9200 - val_loss: 2.5973 - val_accuracy: 0.5701
Epoch 6/10
14/14 [==============================] - 34s 2s/step - loss: 0.0647 - accuracy: 0.9882 - val_loss: 1.8184 - val_accuracy: 0.5888
Epoch 7/10
14/14 [==============================] - 33s 2s/step - loss: 0.0420 - accuracy: 0.9859 - val_loss: 2.1510 - val_accuracy: 0.5981
Epoch 8/10
14/14 

In [75]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

4/4 [==============================] - 7s 2s/step - loss: 1.6462 - accuracy: 0.6822
Test accuracy: 68.22%


In [76]:
# Save the model
#model.save('custom_resnet50_model.h5')

In [77]:
image, labels = load_and_preprocess_images(['images1.jpg'])
    
# Predict probabilities
preds = model.predict(image)
    
# Get the predicted class label
pred_class = np.argmax(preds)
    
# Map predicted class index to class label
pred_label = label_encoder.inverse_transform([pred_class])[0]

print(pred_label)

1/1 [==============================] - 1s 1s/step
hand lump
